In [154]:
import math
import struct
import random
import time

In [53]:
class GeoPoint(object):
    def __init__(self, lat, lon):
        self.lat = lat
        self.lon = lon
        assert -90 <= self.lat <= 90
        assert -180 <= self.lat <= 180
       
    @classmethod
    def from_latlon(cls, latlon):
        parts = latlon.split(',', 1)
        lat = float(parts[0])
        lon = float(parts[1])
        return GeoPoint(lat, lon)
    
    def __repr__(self):
        return '({},{})'.format(self.lat, self.lon)
    
    def __hash__(self):
         return '({},{})'.format(self.lat, self.lon)
        
def read_exactly(f, n_bytes):
    data = b""
    remaining = n_bytes
    while remaining > 0:
        newdata = f.read(remaining)
        data += newdata
        remaining -= len(newdata)
    return data

def interp_integer_2d_grid(x, y, z_as_list):
    z = [[z_as_list[0], z_as_list[1]], [z_as_list[2], z_as_list[3]]]
    x = x % 1
    y = y % 1
    return z[0][0] * (1-x) * (1-y) + z[1][0] * x * (1-y) + z[0][1] * (1-x) * y + z[1][1] * x * y

In [160]:
# Read input.
def get_elevation(raw_locations):
    start = time.time()
    
    
    # Convert to proper geo points.
    locations = raw_locations.split('|')
    for i, loc in enumerate(locations):
        if loc == '':
            raise Exception('Empty location provided in index {}'.format(i))
        try:
            locations[i] = GeoPoint.from_latlon(loc)
        except Exception as e:
            raise Exception('Couldn\'t parse lat,lon string "{}" in index {}'.format(loc, i))
    print('parse', time.time() - start)
    # etopo1 params.
    n_cols = 21601
    n_rows = 10801
    cell_size = 1/60
    bytes_per_cell = 2
    struct_fmt = 'h'
    bin_path = './data/ETOPO1/etopo1_ice_g_i2.bin'

    # For each point, find the correspoinding grid coord, and the neighbouring ones.
    target_coords = []
    interp_coords = []
    interp_coords_flat = []
    nsew = []
    for point in locations:
        grid_x = (point.lon + 180 + cell_size/2) / (360 + cell_size) * n_cols
        grid_y = (90 - point.lat  + cell_size/2) / (180 + cell_size) * n_rows
        target_coords.append((grid_x, grid_y))

        # Compass positions.
        w = math.floor(grid_x)
        e = math.ceil(grid_x)
        n = math.floor(grid_y)
        s = math.ceil(grid_y)

        # Bounds check.
        w = max(w, 0)
        n = max(n, 0)
        e = min(e, n_cols-1)
        s = min(s, n_rows-1)

        # Sanity check.
        if w == e and w > 0:
            w = w-1
        elif w == e and w == 0:
            e = 1
        if n == s and n > 0:
            n = n-1
        elif n == s and n == 0:
            s = 1

        # Add the data.
#         nsew.append((n, s, e, w))
        interp_coords.append([(n,w), (n,e), (s,w), (s,e)])
        interp_coords_flat += [(n,w), (n,e), (s,w), (s,e)]

    print('Coords', time.time() - start)
        
    # Read requried data from file.
    interp_z = {}
    with open(bin_path, 'rb') as f:
        for coord in sorted(set(interp_coords_flat)):
            n_skip_cells = coord[0] * n_cols + coord[1]
            n_skip_bytes = n_skip_cells * bytes_per_cell
            f.seek(n_skip_bytes, 0)
            interp_z[coord] = read_exactly(f, bytes_per_cell)
            
    print('Read', time.time() - start)

    # Process the structs.
    interp_z = {k: struct.unpack('h', v)[0] for k, v in interp_z.items()}
    print('Struct', time.time() - start)

    # Go through each point.
    elevations = []
    for target, interp in zip(target_coords, interp_coords):
        z = [interp_z.get(c) for c in interp]
        elevation = interp_integer_2d_grid(target[0], target[1], z)
        elevations.append(elevation)
    print('Interp', time.time() - start)
    return elevations


# raw_locations = '37.324529,-122.170777'
# print(get_elevation(raw_locations))

In [177]:
raw_locations = '|'.join('{:.5f},{:.5f}'.format(random.uniform(-90, 90), random.uniform(-180, 180)) for _ in range(10))
get_elevation(raw_locations);

parse 0.0
Coords 0.0
Read 0.24779605865478516
Struct 0.24779605865478516
Interp 0.24779605865478516
